This notebook loads the output of the searchlight analyses run using the scripts in [`code/scripts/searchlights`](https://github.com/ContextLab/sherlock-topic-model-paper/tree/master/code/scripts/searchlights). The fMRI data used in the searchlight analyses can be downloaded using the script at [`code/scripts/download_neural_data.sh`](https://github.com/ContextLab/sherlock-topic-model-paper/blob/master/code/scripts/download_neural_data.sh).

## Import libraries

In [1]:
from warnings import filterwarnings

import numpy as np
from nilearn import datasets, surface
from nilearn.image import concat_imgs, load_img, new_img_like
from scipy.stats import ttest_1samp as ttest
from tqdm.notebook import tqdm, trange

from sherlock_helpers.constants import DATA_DIR
from sherlock_helpers.functions import r2z

Helper functions and variables used across multiple notebooks can be found in `/mnt/code/sherlock_helpers/sherlock_helpers`, or on GitHub, [here](https://github.com/ContextLab/sherlock-topic-model-paper/tree/master/code/sherlock_helpers).<br />You can also view source code directly from the notebook with:<br /><pre>    from sherlock_helpers.functions import show_source<br />    show_source(foo)</pre>

## Set paths

In [2]:
sl_vdir = DATA_DIR.joinpath('searchlight_video')
sl_rdir = DATA_DIR.joinpath('searchlight_recall')

## Filter some harmless warnings

In [3]:
# masked voxels in brain data are filled with nans, causes RuntimeWarnings
filterwarnings('ignore', category=RuntimeWarning)

## Load in permutations

In [4]:
video_perms = []
recall_perms = []
for perm in trange(100, leave=False):
    video_perm = load_img(str(sl_vdir.joinpath(f'perm{perm}.nii.gz')))
    recall_perm = load_img(str(sl_rdir.joinpath(f'perm{perm}.nii.gz')))
    video_perms.append(video_perm)
    recall_perms.append(recall_perm)
    
video_perms = concat_imgs(video_perms).dataobj.astype(np.float64)
recall_perms = concat_imgs(recall_perms).dataobj.astype(np.float64)

## Load in real data

In [5]:
ref_img = load_img(str(sl_vdir.joinpath('ref_img.nii.gz')))
                   
vid_imgs = []
rec_imgs = []
for sub in trange(1, 18, leave=False):
    sub_vdata = np.load(sl_vdir.joinpath(f'sub{sub}.npy'), allow_pickle=True)
    sub_rdata = np.load(sl_rdir.joinpath(f'sub{sub}.npy'), allow_pickle=True)
    vid_img = new_img_like(ref_img, sub_vdata.astype(np.float64))
    rec_img = new_img_like(ref_img, sub_rdata.astype(np.float64))
    vid_imgs.append(vid_img)
    rec_imgs.append(rec_img)
    
vid_imgs = concat_imgs(vid_imgs)
rec_imgs = concat_imgs(rec_imgs)

## Get stats for real data

In [6]:
video_data = vid_imgs.dataobj.astype(np.float64)
video_statmap = ttest(np.moveaxis(r2z(video_data), -1, 0), 0).statistic
video_img = new_img_like(ref_img, video_statmap.astype(np.float64))


recall_data = rec_imgs.dataobj.astype(np.float64)
recall_statmap = ttest(np.moveaxis(r2z(recall_data), -1, 0), 0).statistic
recall_img = new_img_like(ref_img, recall_statmap.astype(np.float64))

## Do permutation correction

In [30]:
real_video = video_img.dataobj.astype(np.float64)
real_recall = recall_img.dataobj.astype(np.float64)

zval_video = (real_video - np.nanmean(video_perms, axis=3)) / np.nanstd(video_perms, axis=3)
pval_video = (real_video[:, :, :, np.newaxis] < video_perms).sum(axis=3) / 100
zval_recall = (real_recall - np.nanmean(recall_perms, axis=3)) / np.nanstd(recall_perms, axis=3)
pval_recall = (real_recall[:, :, :, np.newaxis] < recall_perms).sum(axis=3) / 100

zval_video = np.nan_to_num(zval_video)
pval_video = np.nan_to_num(pval_video)
zval_recall = np.nan_to_num(zval_recall)
pval_recall = np.nan_to_num(pval_recall)

## Export unthresholded z-score maps for neurosynth decoding

In [31]:
zmap_video = new_img_like(ref_img, zval_video.astype(np.float64))
zmap_recall = new_img_like(ref_img, zval_recall.astype(np.float64))

# zmap_video.to_filename(str(sl_vdir.joinpath('zmap_video.nii.gz')))
# zmap_recall.to_filename(str(sl_rdir.joinpath('zmap_recall.nii.gz')))

## Threshold

In [9]:
zval_video[pval_video > .05] = 0
zval_video[zval_video < 0] = 0

zval_recall[pval_recall > .05] = 0
zval_recall[zval_recall < 0] = 0

zmap_video = new_img_like(ref_img, zval_video.astype(np.float64))
zmap_recall = new_img_like(ref_img, zval_recall.astype(np.float64))

## Convert to surface maps

In [10]:
fsaverage = datasets.fetch_surf_fsaverage(mesh='fsaverage5')
vid_texture_pl = surface.vol_to_surf(zmap_video, fsaverage.pial_left)
vid_texture_pr = surface.vol_to_surf(zmap_video, fsaverage.pial_right)
rec_texture_pl = surface.vol_to_surf(zmap_recall, fsaverage.pial_left)
rec_texture_pr = surface.vol_to_surf(zmap_recall, fsaverage.pial_right)

## Export for plotting

In [11]:
# np.save(sl_vdir.joinpath('video_surface_left.npy'), vid_texture_pl)
# np.save(sl_vdir.joinpath('video_surface_right.npy'), vid_texture_pr)
# np.save(sl_rdir.joinpath('recall_surface_left.npy'), rec_texture_pl)
# np.save(sl_rdir.joinpath('recall_surface_right.npy'), rec_texture_pr)